In [ ]:
# Load required libraries
library(glmnet)
library(caret)

# Function to load data
load_data <- function(data_path) {
  data <- read.csv(data_path)
  return(data)
}

# Function for data standardization
standardize_data <- function(data) {
  data_scaled <- scale(data)
  return(data_scaled)
}

# Function to calculate metrics
calculate_metrics <- function(true_values, predicted_values) {
  # Calculate metrics (e.g., accuracy, precision, recall, etc.)
  # Example: accuracy <- mean(true_values == predicted_values)
  # Return metrics
}

# Function for k-fold cross-validation
perform_k_fold_cv <- function(data, k) {
  # Define k-fold indices
  folds <- createFolds(data$target_variable, k = k, list = TRUE)
  
  # Initialize vectors to store performance metrics
  # metrics_list <- list()
  
  # Perform k-fold cross-validation
  for (i in 1:k) {
    # Split data into training and testing sets based on fold indices
    train_indices <- unlist(folds[-i])
    test_indices <- unlist(folds[i])
    train_data <- data[train_indices, ]
    test_data <- data[test_indices, ]
    
    # Train elastic net model using glmnet
    # Example:
    # model <- glmnet(x = as.matrix(train_data[, -c("target_variable")]), 
    #                 y = train_data$target_variable, 
    #                 alpha = 0.5, lambda = 0.1)
    
    # Make predictions on the test set
    # predicted_values <- predict(model, newx = as.matrix(test_data[, -c("target_variable")]))
    
    # Calculate performance metrics
    # metrics <- calculate_metrics(test_data$target_variable, predicted_values)
    
    # Store metrics for each fold
    # metrics_list[[i]] <- metrics
  }
  
  # Return performance metrics for all folds
  # return(metrics_list)
}

# Example usage:
# Load data
# data <- load_data("your_data.csv")

# Standardize data
# standardized_data <- standardize_data(data)

# Perform k-fold cross-validation
# k_fold_metrics <- perform_k_fold_cv(standardized_data, k = 5)

# Average and report performance metrics
# mean_metrics <- lapply(k_fold_metrics, function(x) sapply(x, mean))
# print(mean_metrics)


In [ ]:
library(caret)
library(glmnet)

# Define your training control with stratified cross-validation
ctrl <- trainControl(method = "cv",   # Cross-validation method
                     number = k,      # Number of folds (k)
                     classProbs = TRUE,  # Required for stratification
                     summaryFunction = twoClassSummary,  # For binary classification
                     stratify = TRUE)  # Enable stratified sampling

# Train your model using train function with glmnet
model <- train(y ~ .,                  # Formula defining your response variable and predictors
               data = your_data,      # Your dataset
               method = "glmnet",    # Specify "glmnet" for logistic regression with regularization
               trControl = ctrl,     # Use the defined control
               tuneGrid = expand.grid(alpha = 0:1,  # 0 for ridge, 1 for lasso
                                        lambda = seq(0.001, 1, length = 100)))  # Regularization parameter grid

# Access the model performance metrics
summary(model)

# Plot the cross-validation results
plot(model)


# Mimicking paper model (w/o stratified cross validation)

The data in the initial sample (n = 521) were randomly split into 60% training (n = 313), 20% validation (n = 104), and 20% testing data (n = 104) while keeping the proportion of lower IQ consistent across training, validation and testing sets (Table 1). The model training was conducted in the training set, the validation set was used to select the best performing model, and the testing set was used to decide the predictive probability cutoff and model evaluation. An evaluation was also conducted to assess the model performance in the presence of missing values (up to 70%) in an additional independent set (n = 1346; Figure 1, Table 1).

Five models to be tested:
1. elastic-net regularized generalized linear models with no interaction terms (glmnet),
2. support vector machines (svmRadial),
3. random forest (rf),
4. k-Nearest Neighbors (kNN), and
5. gradient boosting (xgbTree)

### data loading and factorisation

In [62]:
library(caret)
library(glmnet)

### DATA LOADING AND SPLITTING
### ------------------------------------------------------

df <- read.csv("ref129_less18_fsiq_1554.csv")

dropped_paras = c('subject_sp_id','derived_cog_impair','asd', 'fsiq', 'fsiq_score', 'fsiq_80')
df <- df[, !(names(df) %in% dropped_paras)]

print("Done.")

[1] "Done."


In [63]:
parameters <- c(
  "q02_catch_ball_2.0", "q02_catch_ball_3.0", "q02_catch_ball_4.0", "q02_catch_ball_5.0",
  "q05_run_fast_similar_1.0", "q05_run_fast_similar_2.0", "q05_run_fast_similar_3.0", "q05_run_fast_similar_4.0", "q05_run_fast_similar_5.0",
  "q06_plan_motor_activity_1.0", "q06_plan_motor_activity_2.0", "q06_plan_motor_activity_3.0", "q06_plan_motor_activity_4.0", "q06_plan_motor_activity_5.0",
  "q09_appropriate_tension_printing_writing_1.0", "q09_appropriate_tension_printing_writing_2.0", "q09_appropriate_tension_printing_writing_3.0", "q09_appropriate_tension_printing_writing_4.0", "q09_appropriate_tension_printing_writing_5.0",
  "q10_cuts_pictures_shapes_1.0", "q10_cuts_pictures_shapes_2.0", "q10_cuts_pictures_shapes_3.0", "q10_cuts_pictures_shapes_4.0", "q10_cuts_pictures_shapes_5.0",
  "q11_likes_sports_motors_skills_1.0", "q11_likes_sports_motors_skills_2.0", "q11_likes_sports_motors_skills_3.0", "q11_likes_sports_motors_skills_4.0", "q11_likes_sports_motors_skills_5.0",
  "q13_quick_competent_tidying_up_1.0", "q13_quick_competent_tidying_up_2.0", "q13_quick_competent_tidying_up_3.0", "q13_quick_competent_tidying_up_4.0", "q13_quick_competent_tidying_up_5.0",
  "q01_whole_body_0.0", "q01_whole_body_1.0", "q01_whole_body_2.0", "q01_whole_body_3.0",
  "q03_hand_finger_0.0", "q03_hand_finger_1.0", "q03_hand_finger_2.0", "q03_hand_finger_3.0",
  "q07_hits_self_body_0.0", "q07_hits_self_body_1.0", "q07_hits_self_body_2.0", "q07_hits_self_body_3.0",
  "q08_hits_self_against_object_0.0", "q08_hits_self_against_object_1.0", "q08_hits_self_against_object_2.0", "q08_hits_self_against_object_3.0",
  "q09_hits_self_with_object_0.0", "q09_hits_self_with_object_1.0", "q09_hits_self_with_object_2.0", "q09_hits_self_with_object_3.0",
  "q12_rubs_0.0", "q12_rubs_1.0", "q12_rubs_2.0", "q12_rubs_3.0",
  "q16_complete_0.0", "q16_complete_1.0", "q16_complete_2.0", "q16_complete_3.0",
  "q18_checking_0.0", "q18_checking_1.0", "q18_checking_2.0", "q18_checking_3.0",
  "q19_counting_0.0", "q19_counting_1.0", "q19_counting_2.0", "q19_counting_3.0",
  "q22_touch_tap_0.0", "q22_touch_tap_1.0", "q22_touch_tap_2.0", "q22_touch_tap_3.0",
  "q32_insists_walking_0.0", "q32_insists_walking_1.0", "q32_insists_walking_2.0", "q32_insists_walking_3.0",
  "q27_play_0.0", "q27_play_1.0", "q27_play_2.0", "q27_play_3.0",
  "q28_communication_0.0", "q28_communication_1.0", "q28_communication_2.0", "q28_communication_3.0",
  "q29_things_same_place_0.0", "q29_things_same_place_1.0", "q29_things_same_place_2.0", "q29_things_same_place_3.0",
  "q31_becomes_upset_0.0", "q31_becomes_upset_1.0", "q31_becomes_upset_2.0", "q31_becomes_upset_3.0",
  "q34_dislikes_changes_0.0", "q34_dislikes_changes_1.0", "q34_dislikes_changes_2.0", "q34_dislikes_changes_3.0",
  "q35_insists_door_0.0", "q35_insists_door_1.0", "q35_insists_door_2.0", "q35_insists_door_3.0",
  "q36_likes_piece_music_0.0", "q36_likes_piece_music_1.0", "q36_likes_piece_music_2.0", "q36_likes_piece_music_3.0",
  "q39_insists_time_0.0", "q39_insists_time_1.0", "q39_insists_time_2.0", "q39_insists_time_3.0",
  "q41_strongly_attached_0.0", "q41_strongly_attached_1.0", "q41_strongly_attached_2.0", "q41_strongly_attached_3.0",
  "q43_fascination_movement_0.0", "q43_fascination_movement_1.0", "q43_fascination_movement_2.0"
)


df[parameters] <- lapply(df[parameters], function(x) as.numeric(as.logical(x)))

print("Done.")

[1] "Done."


### data splitting

In [64]:
# Set seed for reproducibility
set.seed(123)

# Create a vector of indices for stratified sampling
train_index <- createDataPartition(df$fsiq_70, p = 0.6, list = FALSE, times = 1)
remaining_data <- df[-train_index, ]

# Now create test and validation sets from the remaining data
test_index <- createDataPartition(remaining_data$fsiq_70, p = 0.5, list = FALSE, times = 1)
valid_index <- setdiff(seq(nrow(remaining_data)), test_index)

# Split the dataset
train_data <- df[train_index, ]
valid_data <- df[valid_index, ]
test_data <- df[test_index, ]

print("Done.")

[1] "Done."


In [65]:
dim(train_data)
dim(valid_data)
dim(test_data)

[1] 933 224

[1] 310 224

[1] 311 224

### minmax scaling

In [59]:
### MINMAX SCALING
### ------------------------------------------------------

# Define the parameters for Min-Max scaling
parameters_to_scale <- c("age_onset_mos", "fed_self_spoon_age_mos", "smiled_age_mos", "fine_motor_handwriting")  

# Perform Min-Max scaling on the training set
train_data_orig <- train_data  # Make a copy of the training data
train_data[, parameters_to_scale] <- apply(train_data[, parameters_to_scale], 2, function(x) (x - min(x)) / (max(x) - min(x)))

# Perform Min-Max scaling on the validation set
valid_data_orig <- valid_data  # Make a copy of the validation data
valid_data[, parameters_to_scale] <- apply(valid_data[, parameters_to_scale], 2, function(x) (x - min(x)) / (max(x) - min(x)))

# Perform Min-Max scaling on the testing set
test_data_orig <- test_data  # Make a copy of the testing data
test_data[, parameters_to_scale] <- apply(test_data[, parameters_to_scale], 2, function(x) (x - min(x)) / (max(x) - min(x)))

print("Done.")

[1] "Done."


### drop parameters + extract X and y

In [60]:
# For train set
X_train <- train_data[, !(names(train_data) %in% "fsiq_70")]
y_train <- train_data$fsiq_70
y_train <- factor(y_train, levels = c("0", "1"))  # factorise

# For validation set
X_valid <- valid_data[, !(names(valid_data) %in% "fsiq_70")]
y_valid <- valid_data$fsiq_70
y_train <- factor(y_train, levels = c("0", "1"))  # factorise


# For test set
X_test <- test_data[, !(names(test_data) %in% "fsiq_70")]
y_test <- test_data$fsiq_70
y_train <- factor(y_train, levels = c("0", "1"))  # factorise


print("Done.")

[1] "Done."


In [44]:
library(Matrix) 

# Model Training
model_glmnet <- train(y_train ~ ., data = X_train, method = "glmnet")
model_svmRadial <- train(y_train ~ ., data = X_train, method = "svmRadial")
model_rf <- train(y_train ~ ., data = X_train, method = "rf")
model_kNN <- train(y_train ~ ., data = X_train, method = "knn")
model_xgbTree <- train(y_train ~ ., data = X_train, method = "xgbTree")

print("Done.")

Warning message in read.dcf(file.path(p, "DESCRIPTION"), c("Package", "Version")):
"cannot open compressed file 'D:/SOFTWARE/R-4.3.1/library/Matrix/DESCRIPTION', probable reason 'No such file or directory'"


ERROR: Error: Required packages are missing: Matrix


In [ ]:
# Validation set -> to select the best performing model
valid_perf_glmnet <- predict(model_glmnet, newx = as.matrix(X_valid), type = "response")
valid_perf_svmRadial <- predict(model_svmRadial, newdata = X_valid)
valid_perf_rf <- predict(model_rf, newdata = X_valid)
#valid_perf_kNN <- predict(model_kNN, newdata = X_test, type = "prob")
valid_perf_kNN <- predict(model_kNN, newdata = as.matrix(X_valid), type = "prob")
valid_perf_xgbTree <- predict(model_xgbTree, newdata = as.matrix(X_valid))

print("Validation done.")

# Testing set -> to decide the predictive probability cutoff and model evaluation
test_perf_glmnet <- predict(model_glmnet, newx = as.matrix(X_test), type = "response")
test_perf_svmRadial <- predict(model_svmRadial, newdata = X_test)
test_perf_rf <- predict(model_rf, newdata = X_test)
# test_perf_kNN <- knn(train = as.matrix(X_train), test = as.matrix(X_test), cl = y_test)
test_perf_xgbTree <- predict(model_xgbTree, newdata = as.matrix(X_test))

print("Testing done.")

In [ ]:


# Independent set


print("Done.")

In [ ]:
# Function to calculate F1-score
calculate_f1_score <- function(true_labels, predicted_labels) {
  confusion_matrix <- table(true_labels, predicted_labels)
  precision <- confusion_matrix[2, 2] / sum(confusion_matrix[, 2])
  recall <- confusion_matrix[2, 2] / sum(confusion_matrix[2, ])
  f1_score <- 2 * precision * recall / (precision + recall)
  return(f1_score)
}

# Function to calculate accuracy, AUC, and F1-score
calculate_metrics <- function(true_labels, predicted_probs, cutoff) {
  predicted_labels <- ifelse(predicted_probs > cutoff, 1, 0)
  accuracy <- mean(predicted_labels == true_labels)
  auc <- AUC(prediction(predicted_probs, true_labels))
  f1_score <- calculate_f1_score(true_labels, predicted_labels)
  return(list(accuracy = accuracy, auc = auc, f1_score = f1_score))
}


# Determine the predictive probability cutoff for each model
cutoff_glmnet <- optimalCutoff(test_perf_glmnet$obs, test_perf_glmnet$glmnet, opt.methods = "youden")
cutoff_svmRadial <- optimalCutoff(test_perf_svmRadial$obs, test_perf_svmRadial$svmRadial, opt.methods = "youden")
cutoff_rf <- optimalCutoff(test_perf_rf$obs, test_perf_rf$rf, opt.methods = "youden")
#cutoff_kNN <- optimalCutoff(test_perf_kNN$obs, test_perf_kNN$kNN, opt.methods = "youden")
cutoff_xgbTree <- optimalCutoff(test_perf_xgbTree$obs, test_perf_xgbTree$xgbTree, opt.methods = "youden")



# Assess model performance on the testing set
metrics_glmnet <- calculate_metrics(test_perf_glmnet$obs, test_perf_glmnet$glmnet, cutoff_glmnet)
metrics_svmRadial <- calculate_metrics(test_perf_svmRadial$obs, test_perf_svmRadial$svmRadial, cutoff_svmRadial)
metrics_rf <- calculate_metrics(test_perf_rf$obs, test_perf_rf$rf, cutoff_rf)
# metrics_kNN <- calculate_metrics(test_perf_kNN$obs, test_perf_kNN$kNN, cutoff_kNN)
metrics_xgbTree <- calculate_metrics(test_perf_xgbTree$obs, test_perf_xgbTree$xgbTree, cutoff_xgbTree)

In [ ]:
# Train the glmnet model using a range of lambda values
lambda_values <- seq(0.001, 1, by = 0.01)  # Define lambda values
auc_scores <- numeric(length(lambda_values))  # Store AUC scores for each lambda

for (i in 1:length(lambda_values)) {
  # Train glmnet model with current lambda value
  model <- glmnet(x = as.matrix(X_train), y = y_train_num, alpha = 0.1, lambda = lambda_values[i])
  
  # Make predictions on validation set
  predictions <- predict(model, newx = as.matrix(X_val), s = lambda_values[i])

  # Extract predicted probabilities for the positive class
  # predictions_vector <- as.vector(final_predictions)
  
  # Calculate AUC
  auc_scores[i] <- roc(y_val_num, predictions)$auc
}

avg_auc <- mean(auc_scores)
print(avg_auc)

# Find the index of the lambda value with the highest AUC
best_lambda_index <- which.max(auc_scores)
best_lambda <- lambda_values[best_lambda_index]

# Train the final model using the best lambda value
final_model <- glmnet(x = as.matrix(X_train), y = y_train_num, alpha = 0.1, lambda = best_lambda)

# Make predictions on test set using the final model
final_predictions <- predict(final_model, newx = as.matrix(X_test), s = best_lambda)

# Extract predicted probabilities for the positive class
final_predictions_vector <- as.vector(final_predictions)

# Calculate AUC on the validation set
final_auc <- roc(y_test_num, final_predictions_vector)$auc

# Report the AUC score
print(final_auc)

alpha 0.1 
    AUC test = 0.9 
    AUC val = 0.8535

alpha 0.5 AUC = 0.8972

In [ ]:
lambda_values <- seq(0.001, 1, by = 0.01)  # Define lambda values
test_auc_scores <- numeric(length(lambda_values))

for (i in 1:length(lambda_values)) {
  # Make predictions on test set
  predictions <- predict(model, newx = as.matrix(X_test), s = lambda_values[i])
  
  # Calculate AUC
  test_auc_scores[i] <- roc(y_test_num, predictions)$auc
}

mean(test_auc_scores)
sd(test_auc_scores)

# Model Comparison

In [ ]:
# Train the glmnet model using a range of lambda values
lambda_values <- seq(0.001, 1, by = 0.01)  # Define lambda values
auc_scores <- numeric(length(lambda_values))  # Store AUC scores for each lambda

for (i in 1:length(lambda_values)) {
  # Train glmnet model with current lambda value
  model <- glmnet(x = as.matrix(X_train), y = y_train_num, alpha = 0.1, lambda = lambda_values[i])
  
  # Make predictions on validation set
  predictions <- predict(model, newx = as.matrix(X_val), s = lambda_values[i])

  # Extract predicted probabilities for the positive class
  # predictions_vector <- as.vector(final_predictions)
  
  # Calculate AUC
  auc_scores[i] <- roc(y_val_num, predictions)$auc
}

avg_auc <- mean(auc_scores)
print(avg_auc)

# Find the index of the lambda value with the highest AUC
best_lambda_index <- which.max(auc_scores)
best_lambda <- lambda_values[best_lambda_index]

# Train the final model using the best lambda value
final_model <- glmnet(x = as.matrix(X_train), y = y_train_num, alpha = 0.1, lambda = best_lambda)

# Make predictions on test set using the final model
final_predictions <- predict(final_model, newx = as.matrix(X_test), s = best_lambda)

# Extract predicted probabilities for the positive class
final_predictions_vector <- as.vector(final_predictions)

# Calculate AUC on the validation set
final_auc <- roc(y_test_num, final_predictions_vector)$auc

# Report the AUC score
print(final_auc)

### sensitivity, specificity, confusion matrix

In [ ]:
library(caret)

cutoff = 0.5

# Create confusion matrix
conf_matrix <- confusionMatrix(data = as.factor(ifelse(final_predictions_vector > cutoff, 1, 0)),
                               reference = as.factor(ifelse(y_test_num == 1, 1, 0)))

conf_matrix

## predictive probability determination

In [ ]:
# Make predictions on test set using the final model
final_predictions <- predict(final_model, newx = as.matrix(X_test), s = best_lambda)
final_predictions_vector <- as.vector(final_predictions)

cutoff_values <- seq(0.1, 0.9, by = 0.01)

# Initialize vectors 
sensitivity <- numeric(length(cutoff_values))
specificity <- numeric(length(cutoff_values))
accuracy <- numeric(length(cutoff_values))

# Iterate over the range of cutoff values
for (i in seq_along(cutoff_values)) {
  # Convert predicted probabilities to binary predictions using the current cutoff
  binary_predictions <- ifelse(final_predictions_vector > cutoff_values[i], 1, 0)
  
  # Compute confusion matrix
  confusion <- table(binary_predictions, y_test_num)
  
  # Extract true positives, false positives, true negatives, false negatives
  TP <- confusion[2, 2]
  FP <- confusion[2, 1]
  TN <- confusion[1, 1]
  FN <- confusion[1, 2]
  
  # Calculate sensitivity, specificity, accuracy
  sensitivity[i] <- TP / (TP + FN)
  specificity[i] <- TN / (TN + FP)
  accuracy[i] <- (TP + TN) / sum(confusion)
}

# Choose the cutoff with the maximum combined sensitivity and specificity
combined_performance <- sensitivity + specificity
best_cutoff_index <- which.max(combined_performance)
best_cutoff <- cutoff_values[best_cutoff_index]

# Use the best cutoff to compute the confusion matrix
conf_matrix <- confusionMatrix(data = as.factor(ifelse(final_predictions_vector > best_cutoff, 1, 0)),
                               reference = as.factor(ifelse(y_test_num == 1, 1, 0)))

# Print the confusion matrix
print("Confusion Matrix:")
print(conf_matrix)
